In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pulp 
from tabulate import tabulate

In [2]:
import torch 
import torch.nn as nn 

from hawq.utils.quantization_utils.quant_utils import SymmetricQuantFunction, symmetric_linear_quantization_params

from models.three_layer import ThreeLayer_BN, ThreeLayerMLP
from models.q_three_layer import QThreeLayer_BN

from utils.calc_bops import calc_hawq_bops

/home/jcampos/miniforge3/envs/hawq-test-env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import os
import sys 

import torch
import torch.nn as nn

from hawq.utils.export import ExportManager

from models.three_layer import ThreeLayerMLP
from models.q_three_layer import QThreeLayer
from utils.train_utils import config_model

## 1. Setup

In [4]:
model = ThreeLayerMLP()
model.load_state_dict(torch.load("/data/jcampos/hawq-jet-tagging/checkpoints/model_best.pth.tar")["state_dict"])

<All keys matched successfully>

In [5]:
# version 4
# calculate the L2 norm.
# based on HAWQv2 appendix A (symmetric quantization z=0)

MIN_BITWIDTH = 4
MAX_BITWIDTH = 8
LAYERS = ["dense_1","dense_2","dense_3","dense_4"]  # for jettagger MLP

all_bit_widths = list(range(MIN_BITWIDTH,MAX_BITWIDTH+1))
delta_weights = {}  # store the L2 norm. 

for bit_width in all_bit_widths:
    tmp_delta_weights = []
    
    for idx, layer_name in enumerate(LAYERS):
        # get layer to compute quant. error 
        layer = getattr(model, layer_name)
        
        # min and max for chosen bitwidth 
        q_min = -(2**bit_width)
        q_max = (2**bit_width)-1

        # quantize and dequantize weights
        x = torch.clamp(layer.weight, q_min, q_max)
        delta = (q_max-q_min)/(q_max-1)
        x_integer = torch.round((x-q_min)/delta)
        x = x_integer*delta+q_min

        # compute the L2 norm. 
        l2_weight_perturbation = ((x.reshape(1,-1) - layer.weight.reshape(1,-1))**2).sum()
        l2_weight_perturbation = l2_weight_perturbation.detach().numpy().item()
        
        # store resut
        tmp_delta_weights.append(l2_weight_perturbation)

    delta_weights[bit_width] = np.array(tmp_delta_weights)


for bit_width in all_bit_widths:
    print(f"{bit_width} {delta_weights[bit_width]}")
    # print(f"{bit_width} {delta_weights[bit_width]/np.array([ 1088, 2080, 1056, 165]).sum()}")

4 [303.83422852 697.54553223 361.45074463  55.21582031]
5 [300.52352905 690.93457031 358.4932251   54.71564865]
6 [298.87918091 687.21173096 356.55566406  54.51513672]
7 [297.98114014 685.22906494 355.17990112  54.42643738]
8 [297.51644897 684.15856934 354.45861816  54.38290024]


In [6]:
# convert to numpy array 
l2_quant_pert = []

for bit_width in delta_weights.keys():
    l2_quant_pert.append(delta_weights[bit_width])

l2_quant_pert = np.array(l2_quant_pert)
print(l2_quant_pert)

[[303.83422852 697.54553223 361.45074463  55.21582031]
 [300.52352905 690.93457031 358.4932251   54.71564865]
 [298.87918091 687.21173096 356.55566406  54.51513672]
 [297.98114014 685.22906494 355.17990112  54.42643738]
 [297.51644897 684.15856934 354.45861816  54.38290024]]


## 2.0 - Integer Linear Programming 

In [7]:
class args():
    def __init__(self):
        pass

args = args
args.model_size_limit = 0.5 
args.bops_limit = 0.0003 
args.latency_limit = 0.5 

In [8]:
# Number of layers (for ILP setup)
NUM_LAYERS = 4

# number of paramers of each layer
parameters = np.array([ 1088, 2080, 1056, 165])

# Hutchinson_trace means the trace of Hessian for each weight matrix.
# These values are already normlized, i.e., Trace / # arameters
Hutchinson_trace = np.array([6.451294, 0.8976602, 0.18409997, 0.08547983])  # original (not normalized)

# BOPs of each layer
bops_2bit = np.array([55296, 47104, 47104, 3520])
bops_3bit = np.array([72704, 67584, 32768, 5120])
bops_4bit = np.array([90112, 92160, 45056, 7040]) 
bops_5bit = np.array([107520, 120832, 59392, 9280])
bops_6bit = np.array([124928, 153600, 75776, 11840])
bops_7bit = np.array([142336, 190464, 94208, 14720])
bops_8bit = np.array([159744, 231424, 114688, 17920])
bops_9bit = np.array([340992, 215040, 106496, 16640])
bops_10bit = np.array([374784, 258048, 128000, 20000])
bops_32bit = np.array([1118208, 2240512, 831119232, 174880])

# bops = np.array([bops_2bit, bops_3bit, bops_4bit, bops_5bit, bops_6bit, bops_7bit, bops_8bit, bops_9bit, bops_10bit]).reshape(-1) / 1024
bops = np.array([bops_4bit, bops_5bit, bops_6bit, bops_7bit, bops_8bit]).reshape(-1) 

In [9]:
# model size 
model_size_32bit = np.sum(parameters) 
# model_size_limit = model_size_32bit * args.model_size_limit

# bops
# BOPS_LIMIT = np.sum(bops_32bit) * args.bops_limit 
BOPS_LIMIT = int(350e3)

### 2.1 - ILP BOPs Size Constrait

In [10]:
# construct the problem 
number_variables = Hutchinson_trace.shape[0]*len(l2_quant_pert) # NUM_LAYERS * BIT_WIDTH_OPTIONS

# first get the variables
variable = {}
for i in range(number_variables):
    variable[f"x{i}"] = LpVariable(f"x{i}", 0, 1, cat=LpInteger)


In [11]:
prob = LpProblem("Model_Size", LpMinimize)

# add objective function, minimize model size 
# prob += sum([variable[f"x{i}"] * parameters[i%4] for i in range(num_variable) ]) <= model_size_limit 

# add objective function, minimize bops 
prob += sum([variable[f"x{i}"] * bops[i] for i in range(number_variables) ]) <= BOPS_LIMIT 

In [12]:
# Each layer has BIT_WIDTH_OPTIONS variables (5 in this case), each variable can either be 0 or 1 
# an extra constraint is needed to chose one bitwidth per layer
prob += sum([variable[f"x{i}"] for i in list(range(0, number_variables, NUM_LAYERS))]) == 1  
prob += sum([variable[f"x{i}"] for i in list(range(1, number_variables, NUM_LAYERS))]) == 1
prob += sum([variable[f"x{i}"] for i in list(range(2, number_variables, NUM_LAYERS))]) == 1
prob += sum([variable[f"x{i}"] for i in list(range(3, number_variables, NUM_LAYERS))]) == 1

In [13]:
prob += sum( [ variable[f"x{i}"] * l2_quant_pert.reshape(-1)[i] * Hutchinson_trace[i%4] for i in range(number_variables) ] ) 

In [14]:
# solve the problem
status = prob.solve(GLPK_CMD(msg=1, options=["--tmlim", "10000","--simplex"]))
# get the result
LpStatus[status]

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/444453f0fe4442c6bc4a1e3f7940e85c-pulp.lp -o /tmp/444453f0fe4442c6bc4a1e3f7940e85c-pulp.sol
 --tmlim 10000 --simplex
Reading problem data from '/tmp/444453f0fe4442c6bc4a1e3f7940e85c-pulp.lp'...
5 rows, 20 columns, 40 non-zeros
20 integer variables, all of which are binary
39 lines were read
GLPK Integer Optimizer 5.0
5 rows, 20 columns, 40 non-zeros
20 integer variables, all of which are binary
Preprocessing...
5 rows, 20 columns, 40 non-zeros
20 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.314e+05  ratio =  2.314e+05
GM: min|aij| =  7.612e-01  max|aij| =  1.314e+00  ratio =  1.726e+00
EQ: min|aij| =  5.805e-01  max|aij| =  1.000e+00  ratio =  1.723e+00
2N: min|aij| =  3.438e-01  max|aij| =  1.000e+00  ratio =  2.909e+00
Constructing initial basis...
Size of triangular part is 5
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
5 rows, 20 columns, 

'Optimal'

In [15]:
# reshape ILP result 
result = []
for i in range(number_variables):
    result.append(value(variable[f"x{i}"]))

result

result = np.array(result).reshape(len(l2_quant_pert),-1)
bitwidth_idxs = np.argmax(result, axis=0)

In [16]:
print(f"Bit Width\tDense1\t\tDense2\t\tDense3\t\tDense4")
for idx in range(len(l2_quant_pert)):
    print(f"{all_bit_widths[idx]:2} \
          {result[idx][0]:4d} \
            {result[idx][1]:3d} \
            {result[idx][2]:3d} \
            {result[idx][3]:3d}")

# get total bops
total_bops = 0
for idx in range(NUM_LAYERS):
    bops_index = bitwidth_idxs[idx]*NUM_LAYERS + idx
    total_bops += bops[bops_index]

print(f"Total BOPs: {total_bops}")
print(f"BOPs limit: {BOPS_LIMIT}")

Bit Width	Dense1		Dense2		Dense3		Dense4
 4              0               0               0               0
 5              0               1               1               1
 6              0               0               0               0
 7              0               0               0               0
 8              1               0               0               0
Total BOPs: 349248
BOPs limit: 350000
